In [ ]:
!pip install -q -r ./FaceRecognition/requirements.txt

In [ ]:
!git clone https://github.com/MohAlbakerKaw/FaceRecognition.git

Cloning into 'FaceRecognition'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 62 (delta 8), reused 25 (delta 2), pack-reused 0
Unpacking objects: 100% (62/62), 2.90 MiB | 13.49 MiB/s, done.


In [ ]:
import cv2, os, datetime, glob
import torch, torchvision
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cascPath = "./FaceRecognition/Configuration_Files/haarcascade_frontalface_default.xml"
face_class = cv2.CascadeClassifier(cascPath)

# Using harrcascade frontal face to extract the face.
def face_extractor(photo):
  # the photo should be in gray scale
  detected = face_class.detectMultiScale(photo)
  if detected == ():
    return None
  else:
    (x, y, w, h) = detected[0]
    cphoto = photo[y:y+h, x:x+w]
    output = cv2.resize(cphoto, (150,150))
  return output

In [ ]:
### TARGETED CLASSES ###
DATA_PATH = './FaceRecognition/Original_Images/'
FOLDERS = os.listdir(DATA_PATH)
print(FOLDERS)

# for folder in FOLDERS:
#     print(FOLDERS)

['Obama', 'Putin', 'Trump']


In [4]:
pwd

In [ ]:
root_dir = './FaceRecognition/Original_Images/'

for folder in FOLDERS:#[:-1]:
    target_dir = './FaceRecognition/Extracted_Faces/' + folder
    os.mkdir(target_dir)
    for filename in glob.iglob(root_dir + folder + '/*.jpg', recursive=True):
        print(filename)
        img = cv2.imread(filename, 0)
        gface = face_extractor(img)
        os.chdir(target_dir)
        # filename
        wfilename = os.path.basename(filename)
        # Using cv2.imwrite() method
        # Saving the image
        cv2.imwrite(wfilename, gface)
        os.chdir(os.path.expanduser('~/sagemaker-studiolab-notebooks'))
    os.chdir(os.path.expanduser('~/sagemaker-studiolab-notebooks'))

In [5]:
# data generator
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [18]:
for folder in FOLDERS:
    for filename in glob.glob('./FaceRecognition/Extracted_Faces/'+folder+'/*.jpg'):
        im = load_img(filename) # this is a Numpy array with shape (3, 150, 150)
        x = img_to_array(im)
        x = x.reshape((1,)+x.shape)
        
        # the .flow() command below generates batches of randomly transformed images
        # and saves the results to the `augmented/` directory
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir='./FaceRecognition/Augmented/'+folder, 
                                  #save_prefix='{}'.format(filename), 
                                  save_format='jpg'):
            i += 1
            if i > 50:
                break  # otherwise the generator would loop indefinitely

In [21]:
### zip folder ###
# !tar chvfz notebook.tar.gz ./FaceRecognition/Extracted_Faces

In [1]:
!pip install split-folders

In [2]:
import splitfolders

In [3]:
folder_path = './FaceRecognition/Augmented'
output_path = './FaceRecognition/Processed'
splitfolders.ratio(folder_path, output_path, seed=42, ratio=(.8, 0.1,0.1)) 

Copying files: 770 files [00:00, 12402.32 files/s]
